## Data wrangling

In [1]:
import pandas as pd
import numpy as np
import glob
import preprocessing as pre

First we need to extract data from the excel file. In order to construct the eventual for loop, we'll first need to try extracting data from one file

In [2]:
df_eeg = pre.process_all_excel_files()

Reorder columns of df

In [3]:
df_eeg = df_eeg[['id', 'brain_oscillation','freq_band','electrode','fft_abs_power']]

Remove participants (10,18, 52 and 215) because of missing Neuropsy data

In [4]:
df_eeg = df_eeg[(df_eeg.id != '10') & (df_eeg.id != '18') & (df_eeg.id != '52') & (df_eeg.id != '215')]

Verify data wrangling

In [5]:
print(df_eeg.shape)
df_eeg.head(5)

(28518, 5)


,id,brain_oscillation,freq_band,electrode,fft_abs_power
0,134,Delta,1.0-4.0Hz,FP1,15.565495
1,134,Delta,1.0-4.0Hz,FP2,13.945462
2,134,Delta,1.0-4.0Hz,F7,10.874886
3,134,Delta,1.0-4.0Hz,F3,13.581803
4,134,Delta,1.0-4.0Hz,Fz,15.644595


In [6]:
df_eeg.dtypes

id                   object
brain_oscillation    object
freq_band            object
electrode            object
fft_abs_power        object
dtype: object

In [7]:
print('# unique ids: {}'.format(len(df_eeg.id.unique())))
print(df_eeg.id.value_counts())

# unique ids: 97
53     294
34     294
702    294
63     294
409    294
      ... 
206    294
212    294
62     294
66     294
25     294
Name: id, Length: 97, dtype: int64


In [8]:
print(df_eeg.electrode.value_counts())
print(df_eeg.brain_oscillation.value_counts())
print(df_eeg.freq_band.value_counts())

Fz     1358
C4     1358
T3     1358
O1     1358
F7     1358
F4     1358
A1     1358
FP1    1358
F3     1358
FP2    1358
Pz     1358
T5     1358
P3     1358
O2     1358
A2     1358
F8     1358
P4     1358
T6     1358
T4     1358
C3     1358
Cz     1358
Name: electrode, dtype: int64
Alpha2       2037
Alpha1       2037
Gamma2       2037
Theta        2037
Gamma1       2037
HighBeta     2037
Beta         2037
Beta1        2037
Beta3        2037
HighGamma    2037
Beta2        2037
Gamma        2037
Delta        2037
Alpha        2037
Name: brain_oscillation, dtype: int64
25.0-30.0Hz    2037
30.0-35.0Hz    2037
10.0-12.0Hz    2037
40.0-50.0Hz    2037
4.0-8.0Hz      2037
12.0-25.0Hz    2037
1.0-4.0Hz      2037
18.0-25.0Hz    2037
12.0-15.0Hz    2037
15.0-18.0Hz    2037
35.0-40.0Hz    2037
30.0-40.0Hz    2037
8.0-10.0Hz     2037
8.0-12.0Hz     2037
Name: freq_band, dtype: int64


### Merge dataframes (Neuropsy data with df (eeg))

We now need to import the Neuropsydata

In [9]:
df_neuropsy = pd.read_excel("Neuropsy.xlsx", na_values=".")
print(df_neuropsy.shape)
df_neuropsy.head(5)

(100, 13)


,ID,Age,Gender,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,21,1,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
1,3,20,1,10.0,5.0,13.0,1.0,27.0,34.0,41.0,25.0,31.0,38.0
2,4,18,1,26.0,17.0,7.0,15.0,93.0,89.0,96.0,90.0,92.0,90.0
3,7,23,1,24.0,8.0,6.0,14.0,86.0,66.0,112.0,94.0,90.0,100.0
4,10,18,1,NaN,NaN,NaN,NaN,98.0,103.0,93.0,92.0,100.0,85.0


Then remove participants (10,18, 52 and 215) because of missing Neuropsy data

In [10]:
df_neuropsy= df_neuropsy.dropna(axis=0, how='any')

Verify it worked

In [11]:
print(df_neuropsy.shape)

(96, 13)


Rename ID variable

In [12]:
#rename id so it can be merged and fft_abs_power
df_neuropsy.rename(columns = {'ID':'id'}, inplace = True) #rename id so it can be merged
df_neuropsy.head(5)

,id,Age,Gender,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,21,1,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
1,3,20,1,10.0,5.0,13.0,1.0,27.0,34.0,41.0,25.0,31.0,38.0
2,4,18,1,26.0,17.0,7.0,15.0,93.0,89.0,96.0,90.0,92.0,90.0
3,7,23,1,24.0,8.0,6.0,14.0,86.0,66.0,112.0,94.0,90.0,100.0
5,11,21,1,16.0,26.0,13.0,10.0,45.0,33.0,78.0,69.0,51.0,94.0


#### Now let's merge the two dataframes together

In [13]:
 #change dtype of id column from df
df_eeg['id']=df_eeg['id'].astype(int)

In [14]:
df_full = pd.merge(left=df_eeg, right=df_neuropsy, left_on='id', right_on='id')

In [15]:
df_full.head()

,id,brain_oscillation,freq_band,electrode,fft_abs_power,Age,Gender,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,134,Delta,1.0-4.0Hz,FP1,15.565495,17,1,18.0,3.0,6.0,6.0,85.0,91.0,81.0,91.0,90.0,94.0
1,134,Delta,1.0-4.0Hz,FP2,13.945462,17,1,18.0,3.0,6.0,6.0,85.0,91.0,81.0,91.0,90.0,94.0
2,134,Delta,1.0-4.0Hz,F7,10.874886,17,1,18.0,3.0,6.0,6.0,85.0,91.0,81.0,91.0,90.0,94.0
3,134,Delta,1.0-4.0Hz,F3,13.581803,17,1,18.0,3.0,6.0,6.0,85.0,91.0,81.0,91.0,90.0,94.0
4,134,Delta,1.0-4.0Hz,Fz,15.644595,17,1,18.0,3.0,6.0,6.0,85.0,91.0,81.0,91.0,90.0,94.0


In [17]:
#sort by ids
df_full = df_full.sort_values(by=['id'])

In [18]:
#Reorder columns 
df_full = df_full[['id','Age', 'Gender','brain_oscillation','freq_band','electrode','fft_abs_power', 'cIM', 'cHR', 'cIE', 'cSC', 'Aqtot', 'Aqaudi', 'Aqvis', 'RCQtot', 'RCQaudi', 'RCQvis']]
df_full.head()

,id,Age,Gender,brain_oscillation,freq_band,electrode,fft_abs_power,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
24987,1,21,1,Gamma2,35.0-40.0Hz,O2,0.307275,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
24702,1,21,1,Delta,1.0-4.0Hz,F8,7.025115,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
24696,1,21,1,Delta,1.0-4.0Hz,FP1,15.376173,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
24697,1,21,1,Delta,1.0-4.0Hz,FP2,16.585338,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0
24698,1,21,1,Delta,1.0-4.0Hz,F7,7.968847,17.0,31.0,29.0,9.0,90.0,91.0,92.0,94.0,80.0,110.0


Adjust data types in dataframe

In [19]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28224 entries, 24987 to 14991
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 28224 non-null  int64  
 1   Age                28224 non-null  int64  
 2   Gender             28224 non-null  int64  
 3   brain_oscillation  28224 non-null  object 
 4   freq_band          28224 non-null  object 
 5   electrode          28224 non-null  object 
 6   fft_abs_power      28224 non-null  object 
 7   cIM                28224 non-null  float64
 8   cHR                28224 non-null  float64
 9   cIE                28224 non-null  float64
 10  cSC                28224 non-null  float64
 11  Aqtot              28224 non-null  float64
 12  Aqaudi             28224 non-null  float64
 13  Aqvis              28224 non-null  float64
 14  RCQtot             28224 non-null  float64
 15  RCQaudi            28224 non-null  float64
 16  RCQvis            

In [38]:
df_full['fft_abs_power'] = df_full['fft_abs_power'].astype(float)
print(df_full.dtypes)

id                     int64
Age                    int64
Gender                 int64
brain_oscillation     object
freq_band             object
electrode             object
fft_abs_power        float64
cIM                  float64
cHR                  float64
cIE                  float64
cSC                  float64
Aqtot                float64
Aqaudi               float64
Aqvis                float64
RCQtot               float64
RCQaudi              float64
RCQvis               float64
dtype: object


## Descriptive statistics

#### Check for missing values

In [39]:
print(df_full.isnull().values.any())
print(df_full.isnull().values.sum())

False
0


#### Descriptive table summary

In [48]:
print(df_full.describe())
df_full.describe()

                 id           Age        Gender  fft_abs_power           cIM  \
count  28224.000000  28224.000000  28224.000000   28224.000000  28224.000000   
mean     161.187500     19.645833      1.406250       5.081187     21.843750   
std      176.477706      2.832081      0.491141       8.846661      6.660524   
min        1.000000     17.000000      1.000000       0.037380      5.000000   
25%       42.750000     18.000000      1.000000       0.832856     18.000000   
50%       82.500000     19.000000      1.000000       2.194313     23.000000   
75%      212.250000     21.000000      2.000000       5.988173     26.250000   
max      702.000000     29.000000      2.000000     258.114724     33.000000   

                cHR           cIE           cSC         Aqtot        Aqaudi  \
count  28224.000000  28224.000000  28224.000000  28224.000000  28224.000000   
mean      17.687500     14.479167      9.645833     81.489583     83.479167   
std        7.606755      6.977735      4.2

,id,Age,Gender,fft_abs_power,cIM,cHR,cIE,cSC,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
count,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000,28224.000000
mean,161.187500,19.645833,1.406250,5.081187,21.843750,17.687500,14.479167,9.645833,81.489583,83.479167,83.302083,71.729167,76.593750,73.614583
std,176.477706,2.832081,0.491141,8.846661,6.660524,7.606755,6.977735,4.240209,26.034891,25.060363,27.709966,24.841384,22.465453,24.461870
min,1.000000,17.000000,1.000000,0.037380,5.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,16.000000,19.000000,8.000000
25%,42.750000,18.000000,1.000000,0.832856,18.000000,12.750000,8.750000,7.000000,70.500000,73.750000,69.750000,55.750000,60.000000,55.750000
50%,82.500000,19.000000,1.000000,2.194313,23.000000,17.000000,15.000000,10.000000,86.500000,89.000000,89.500000,70.500000,79.500000,73.500000
75%,212.250000,21.000000,2.000000,5.988173,26.250000,24.000000,18.000000,12.000000,101.000000,100.250000,104.250000,92.250000,92.500000,94.000000
max,702.000000,29.000000,2.000000,258.114724,33.000000,33.000000,30.000000,18.000000,125.000000,127.000000,121.000000,125.000000,132.000000,122.000000
